# Part I : Quel(le) data scientist êtes-vous ?
## Contexte de l’analyse

Elu métier le plus sexy par la Harvard Business Review en octobre 2012, le data scientist représente un profil rare qui exige de nombreuses compétences.

A partir d'un dataset Aquila, vous réaliserez :
- un clustering non supervisé afin d'identifier 2 groupes de profils techniques distinctes
- une prédiction des profils dont le métier n'est pas labellisé


## Données
data.csv contient 6 variables : 
    - 'Entreprise' correspond à une liste d'entreprises fictive
    - 'Metier' correspond au métier parmi data scientist, lead data scientist, data engineer et data architecte
    - 'Technologies' correspond aux compétences maîtrisées par le profil
    - 'Diplome' correspond à son niveau scolaire (Bac, Master, PhD,...)
    - 'Experience' correspond au nombre d'années d'expériences
    - 'Ville' correspond au lieu de travail
    


## Répondez aux questions 

Bonne chance!

In [1]:
# Import des libraries classique (numpy, pandas, ...)
import pandas as pd
import numpy as np
import re
import sklearn as sk
import seaborn as sb
from matplotlib import pyplot as plt
from sklearn.metrics  import *
%matplotlib inline
plt.style.use('ggplot')


In [2]:
string = 'mayi va appeler cedric au 0752644800'
print(re.findall(r'\d+', string))

['0752644800']


### 1) Importer le tableau de données dans un dataframe 

In [3]:
# Avoir un apercu des donnees, detecter le separateur
# et le caractere utilise pour les virgules
ligne_a_lire = 6
with open('data.csv','r') as file:
    for i in range(ligne_a_lire):
        print(file.readline())

Entreprise,Metier,Technologies,Diplome,Experience,Ville

Sanofi,Data scientist,Matlab/Python/Pyspark/Scikit-learn/Tensorflow,Master,1,Paris

Massachusetts General Hospital(MGH),Data architecte,Python/Java/Scala/MongoDB,Master,3,Marseille

Delve,Lead data scientist,SPSS/SQL/Teradata/R/Python/Tensorflow/scikit-learn,Master,3,Nantes

Ann & Robert H. Lurie Children’s Hospital of Chicago,Data scientist,C/C++/Java/Python,Master,"1,5",Marseille

Arm,Data scientist,Matlab/Python/C++/numpy/Tensorflow/scikit-learn,Phd,,Bordeaux



Le separateur de columne ici est une <b>','</b>. Nous allons l'utiliser avec pandas 

In [6]:
# Import du dataframe "data.csv"
filename = 'data.csv'
df = pd.read_csv(filename, sep=',', decimal=',')

In [ ]:
# Avoir un apercu des premieres lignes du fichier
nombre_lignes_a_visualiser = 162
df.head(nombre_lignes_a_visualiser)

### 2) Combien y a t-il d'observations dans ce dataset? Y a t-il des valeurs manquantes? 

In [ ]:
# methode 1 : avec shape
# methode 2 : avec len
print(' methode 1: nombre d\'observations du dataset {} \n'.format(df.shape[0]))
print(' methode 2: nombre d\'observations du dataset {} \n'.format(len(df)))


In [ ]:
## Apercu des valeurs manquantes
## Calculons la proportion de valeurs manquantes dans chacune des columnes
print('proportion des valeurs manquantes dans chacune des columnes en % \n')
print(df.isnull().mean()*100)

<br> Concernant les valeurs manquantes, je constate que les variables categorielles <b> Entreprise</b>, <b>Metier</b> en contiennent approximativement et respectivement <b>0.17%</b> et <b>0.18%</b> tandis que la variable numerique <b>Experience</b> en contient moins de <b>1%</b>.
<br> Ces donnees manquantes sont assez rares (moins de 1%) en general, le dataset est donc exploitable et on peut remplacer ces valeurs manquantes par diverses méthodes (moyenne, medianne, mode (pour les variables categorielles)

### 3) Réaliser l'imputation des valeurs manquantes pour la variable "Experience" avec : 
- la valeur médiane pour les data scientists
- la valeur moyenne pour les data engineers

In [ ]:
# Realisation de l'imputation de la valeur manquante
## Remplacer les valeurs manquantes pour les data scientists par la valeur mediane
index_data_scientist = list(df[df.Metier.isin(['Data scientist'])].isnull().index.values)
df.loc[index_data_scientist,'Experience'] = df.loc[index_data_scientist,'Experience'].transform(lambda col:col.fillna(col.median()))

## Remplacer les valeurs manquantes pour les data engineers par la moyenne
index_data_engineer = list(df[df.Metier.isin(['Data engineer'])].isnull().index.values)
df.loc[index_data_engineer,'Experience'] = df.loc[index_data_engineer,'Experience'].transform(lambda col:col.fillna(col.mean()))


La méthode que j'ai employee ici, consiste d'abord a detecter <b> les indices </b> du dataset correspondant aux differents emplois qu'on recherche ici, à savoir <b> data scientist </b> et <b> data engineers </b>.
<br> Ensuite nous allons proceder à la modification des <b> NaN </b> presents à ces ligne par les valeurs demandées.
<br> <b> Il existe surement des méthodes plus optimisées mais celle la me parait pas, j'aurai le temps de reflechir à d'autres méthodes et calculer la plus rapide </b>

### 4) Combien d'années d'expériences ont, en moyenne, chacun des profils : le data scientist, le lead data scientist et le data engineer en moyenne?

In [ ]:
# Calcul des annees d'experiences de chacun des profils
print('Moyenne des experiences des differents metiers \n {}'.\
      format(df.groupby('Metier')['Experience'].mean()))


<b> Le data scientist </b> a en moyenne <b>3.0 annees </b> d'experiences. <br>
<b> Le lead data scientist </b> a en moyenne <b>3.96 annees </b> d'experiences. <br>
<b> Le data engineer </b> a en moyenne <b>2.98 annees </b> d'experiences.

### 5) Faire la représentation graphique de votre choix afin de comparer le nombre moyen d'années d'expériences pour chaque métier

In [ ]:
df.groupby('Metier')['Experience'].mean().plot(kind='bar')
plt.title('Annee d\'experience de divers acteurs de la data')

### 6) Transformer la variable continue 'Experience' en une nouvelle variable catégorielle 'Exp_label' à 4 modalités: débutant, confirmé, avancé et expert
- Veuillez expliquer votre choix du règle de transformation.  

On va partir du postulat qu'un lead data scientist est un <b> avance </b> <br> En effet ce dernier est amené à gerer une equipe de data scientist debutants( sortant d'ecoles et de confirmés), la valeur mediane du lead data scientist etant de 3.5 et la moyenne de 3.96, on va considere qu'a partir de 4 ans d'experience, un ingénieur (architecte, data scientist etc...) deviens expert dans son domaine

In [ ]:
def categorized_variable(value):
    '''
    INPUT 
        - value - annees d'experiences
    OUTPUT 
        - categorie choisie
    '''
    if value<2:
        return 'Debutant'
    elif value>=2 and value<4:
        return 'Confirme'
    elif value>=4 and value<=8:
        return 'Avance'
    else:
        return 'Expert'

# Creer une variable categorielle à 4 modalites
df['Exp_label'] = df['Experience'].apply(categorized_variable)

In [ ]:
data = df.Experience.dropna(how='all')
plt.hist(data)

In [ ]:
df.head(5)

### 7) Quelles sont les 5 technologies les plus utilisées? Faites un graphique

Listons l'ensemble des technologies utilisées par l'ensemble des candidats

In [ ]:
# Listons l'ensemble des technologies utilisees
technologies_utilisees = list(map(lambda x:x.split('/'), df.Technologies))
# Mettons à plat (nested) la liste de liste obtenu precedemment
liste_des_logiciels_utilises = [val.lower() for sublist in technologies_utilisees for val in sublist if len(val)!=0]

# Stockons la liste unique des logiciels utilises
liste_unique_des_logiciels_utilises = list(set(liste_des_logiciels_utilises))

In [ ]:
liste_unique_des_logiciels_utilises

In [ ]:
# Nombre des N technologies desirees
liste_n_largest = 5
# Stockons dans une liste le nombre
sizes = [liste_des_logiciels_utilises.count(elt) for elt in liste_unique_des_logiciels_utilises]
# Avoir les index des N technologies les plus utilisees
index = np.argsort(sizes)[-liste_n_largest:]
# Avoir les noms de ces technologies
labels = np.asarray(liste_unique_des_logiciels_utilises, dtype=str)
labels = list(labels[index])
# Regrouper toutes les autres technologies dans une grosse categorie appelee Autres
labels2 = labels
labels.append ('Autres') #rajouter les autres qu'on a pas comptabiliser
sizes2 =np.asarray(sizes)[index]
sizes2 = list(sizes2)
sizes2.append(len(liste_des_logiciels_utilises)-sum(sizes2)) 

fig1, ax1 = plt.subplots()
ax1.pie(sizes2, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=180)
ax1.axis('equal')
plt.show()

##### je vois l'iddee
liste_des_logiciels_utilises2 = np.asarray(liste_des_logiciels_utilises)  
liste_des_logiciels_utilises2[~np.isin(liste_des_logiciels_utilises2, labels2)] = 'Autres'   
data2 = pd.DataFrame(data=liste_des_logiciels_utilises2, columns=['logiciels'])
ax = sb.countplot( y='logiciels', data= data2)

Conclusion <br>Comme on peut le constater sur le camembert, <br> Python vient en première position des logiciels utilisés  par les personnes de la data avec un score de <b>15.3%</b>, suivi de près par le langage R, prisée des statisticiens <b>10.0%</b>, Ensuite viennent des outils de gestion, de gestion de la donnee comme SQL <b>5.9%</b>, Hadoop <b>3.6%</b>. <br> Java lui est moins utilisé comparé à Python et R, cela peut être lié au fait que les outils orientés Machine Learning sont plus rares ou moins vulgarisés (prisés)
<br> On constate aussi que des mots comme Anglais sont presents dans les technologies et il existe plusieurs denominations du meme outil comme Spark (pyspark). Aussi, des technologies mentionnees matplotlib et scikit learn font partie de python qui lui meme est mentionne. Nous avons reglé le probleme de la denomination du meme logiciel <br>exemple:  Hadoop et hadoop </br> en mettant en miniscule le nom des technologies.
<br> En conclusion, il y'a beaucoup de redondances d'informations. Afin d'effectuer une classification ou une possible regression, nous allons proceder a une reduction de la dimensionnalité du dataset

In [ ]:
liste_des_logiciels_utilises2

### 8) Réaliser une méthode de clustering non supervisée de votre choix pour faire apparaître 2 clusters que vous jugerez pertinents. Donnez les caractéristiques de chacun des clusters.
-  Justifier la performance de votre algorithme grace à une métrique.
-  Interpréter votre resultat.  

### Categoriser les technolgies via le one hot encodeur

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
software_maitrise = pd.DataFrame(mlb.fit_transform(df.Technologies.str.split('/')),columns=mlb.classes_, index=df.index)
software_maitrise = software_maitrise.drop('',axis=1)#element non desire issu du split
#software_maitrise = np.asarray(software_maitrise)

In [12]:
software_maitrise.columns

Index(['AI', 'AWS', 'Anglais', 'Big data', 'C', 'C++', 'Cassandra',
       'Deep learning', 'Docker', 'Elasticsearch', 'Excel', 'GNU', 'HBASE',
       'HDFS', 'Hadoop', 'Hadoop(HDFS)', 'Hive', 'Java', 'Kafka', 'Kibana',
       'Linux', 'Machine learning', 'Map-Reduce', 'MariaDB', 'Matlab',
       'Matplotlib', 'Microsoft Azure', 'MongoDB', 'MySQL', 'NoSQ', 'NoSQL',
       'ORACLE', 'PIG', 'Perl', 'PostgreSQL', 'PySpark', 'Pycharm', 'Pyspark',
       'Python', 'R', 'Redshift', 'Ruby', 'SAS', 'SPSS', 'SQL', 'Scala',
       'Scikit-learn', 'Scoring', 'Spark', 'Tableau', 'Tensorflow', 'Teradata',
       'VBA', 'Vertica', 'Windows', 'Yarn', 'anglais', 'machine learning',
       'numpy', 'scikit-learn'],
      dtype='object')

Il s'agira ici de remplacer toute les variables categorielles par des variables numeriques avec le one hot encoding. 
On peut supprimer la variable metier pour cette etude, par ce que le metier correspont approximativement a une certaine categorie de competences techniques plus ou moins defini, l'entreprise aussi ne semble pas être une variable pertinente, de meme que la ville ou l'emploi est effectué
Le nombre d annee d experience peut definir un profil technique on va get dummies la variable cree

In [8]:
software_maitrise

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
job_experiences = np.asarray(pd.get_dummies(df.Exp_label))
diplomes = np.asarray(pd.get_dummies(df.Diplome))
metiers = np.asarray(pd.get)

In [ ]:
dataset_of_clustering = np.concatenate((software_maitrise, job_experiences, diplomes), axis=1)

### Reduction de la dimensionnalite concernant les technologies

In [ ]:
from sklearn.decomposition import TruncatedSVD

percentage_variance_explained = []
for elt in range(1,dataset_of_clustering.shape[1]):
    tsvd = TruncatedSVD(n_components=elt)
    X =  tsvd.fit(dataset_of_clustering).transform(dataset_of_clustering)
    percentage_variance_explained.append(tsvd.explained_variance_ratio_[0:elt].sum())
 
plt.plot(list(range(1, dataset_of_clustering.shape[1])), percentage_variance_explained)

In [ ]:
from sklearn.decomposition import TruncatedSVD

tsvd = TruncatedSVD(n_components=40)
software_maitrise_to_take =  tsvd.fit(dataset_of_clustering).transform(dataset_of_clustering)
print('pourcentage explique {} \n'.format(tsvd.explained_variance_ratio_[0:40].sum()))
 

In [ ]:
from sklearn import metrics
from sklearn.cluster import KMeans, AffinityPropagation, AgglomerativeClustering
from sklearn.metrics import davies_bouldin_score 
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [ ]:
dataset_of_clustering

In [ ]:
kmeans = KMeans(init='k-means++', n_clusters=2, n_init=100)
kmeans.fit(dataset_of_clustering)
preds = kmeans.fit_predict(dataset_of_clustering)

In [ ]:
clustering = AgglomerativeClustering()
clustering.fit(dataset_of_clustering)

In [ ]:
clustering_validity = silhouette_score(dataset_of_clustering, preds)

In [ ]:
clustering_validity

In [ ]:
df['kmean_label'] = kmeans.labels_
df['agglo_label'] = clustering.labels_

In [ ]:
df.head()

### 9) Réaliser la prédiction des métiers manquants dans la base de données par l'algorithme de votre choix
-  Justifier la performance de votre algorithme grace à une métrique.
-  Interpréter votre resultat.  

In [ ]:
metiers_pas_renseigne = df[pd.isnull(df.Metier)].reset_index()


In [ ]:
dataset_base = df.dropna(subset=['Metier'], how='all')

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
train_set

### 10) Décrire de façon détaillée les différentes étapes pour mener à bien un projet data complexe.

Dans le cadre d’une campagne de recrutement vous avez reçu un ensemble de cv qui ne sont pas à jour (le poste actuel n’est pas présent). Deux exemple de CV vous sont donnés en pièce jointe (il n'y a pas de mise en page type). Tous les CV sont au format PDF.

On souhaite connaitre le poste actuel de chaque candidat.

-  Quelles sont les différentes étapes nécessaires à la réalisation d'un tel projet?
-  Pour chaque étape, détaillez les taches à accomplir, les méthodes à suivre, les pistes possibles, les points critiques.
-  Justifiez consciencieusement vos choix
